In [ ]:
### TEMP until 2021-01-18 scb report !!!! ####
### remove the line below above tag when the next scb update has arrived, scheduled 2021-01-18 ###


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

sns.set()

In [ ]:
scb_file = pd.ExcelFile('https://www.scb.se/hitta-statistik/statistik-efter-amne/befolkning/befolkningens-sammansattning/befolkningsstatistik/pong/tabell-och-diagram/preliminar-statistik-over-doda/')


In [ ]:
rap_date = scb_file.parse(sheet_name='Tabell 8',skiprows=6)
rap_date

In [ ]:
rap_date = rap_date.replace(np.nan,0)
rap_date = rap_date.loc[rap_date['DagMånad'] != 0]
unknowns = rap_date.iloc[-1]

rap_date = rap_date[:-1]
unknowns

In [ ]:
# always pick 2020 unknowns from table 1, where 2020 is its own column (avoid getting 2021 entries incl.)

last_unknown = scb_file.parse('Tabell 1',skiprows=6,usecols=[6])
unknowns = unknowns[1:]
unknowns.iloc[-1] = last_unknown.iloc[-1].values[0]
    
unknowns

In [ ]:
unknowns = unknowns.replace(0,method='ffill')
unknowns

In [ ]:
swe_months = ['januari','februari','mars','april','maj','juni',
              'juli','augusti','september','oktober','november','december']

eng_months = ['January','February','March','April','May','June',
             'July','August','September','October','November','December']

swe_2_eng_map = dict(zip(swe_months,eng_months))
    
idx = pd.to_datetime(rap_date['DagMånad'].apply(lambda d : d.replace(d.split(' ')[1],swe_2_eng_map[d.split(' ')[1]])))
rap_date.index = idx
rap_date = rap_date.drop('DagMånad',axis=1)

rap_date = rap_date.loc[:'2020-12-31']

rap_date


In [ ]:
def swe2eng(i,datum):
    
    month_map = {'jan.' : '01',
                'april' : '04',
                'maj' : '05',
                'juni' : '06',
                'juli' : '07',
                'aug.' : '08',
                'sep.' : '09',
                'okt.' : '10',
                'nov.' : '11',
                'dec.' : '12'}
    
    dag,mon = datum.split()
    
    if len(dag) == 1:
        dag = '0' + dag
    
    if i < 39: # 39 reports 2020
        return '2020-' + month_map[mon] + '-' + dag

    else:
        return '2021-' + month_map[mon] + '-' + dag


In [ ]:
cols = [ swe2eng(i,x) for i,x in enumerate(rap_date.columns.values) ]
rap_date.columns = cols
rap_date

In [ ]:
rap_date.sum()

In [ ]:
unknowns.index = cols
unknowns

In [ ]:
rap_date.loc[:'2020-12-31']

In [ ]:
daily_proportions = rap_date / rap_date.sum()
add_ons = daily_proportions * unknowns
rap_date = rap_date + add_ons
rap_date

In [ ]:
rap_date.sum()

In [ ]:
monthly_deaths_by_rep_date = rap_date.resample('M').sum()

### TEMP until 2021-01-18 scb report !!!! ####
#monthly_deaths_by_rep_date['2021-01-18'] = monthly_deaths_by_rep_date['2021-01-11']
###

monthly_deaths_by_rep_date

In [ ]:
monthly_transposed = monthly_deaths_by_rep_date.T
monthly_transposed.index.name='report_date'
monthly_transposed.columns.name = 'month'
monthly_transposed

In [ ]:
dead_diff_last_report_vs_third_after_month_end = np.zeros(12)
all_diffs = np.zeros((len(monthly_transposed.columns),len(monthly_transposed)))

for i,month in enumerate(monthly_transposed.columns):
    mask = pd.to_datetime(monthly_transposed[month].index) > month + pd.DateOffset(weeks=2)
    
    dead_diff_last_report_vs_third_after_month_end[i] = \
    (monthly_transposed[month][mask][-1] - monthly_transposed[month][mask][0])
    
    for j in range(len(monthly_transposed)):
        all_diffs[i,j] = monthly_transposed[month][-1] - monthly_transposed[month][j]    
    
diff_third_report_vs_last = pd.Series(dead_diff_last_report_vs_third_after_month_end,
                                     index=range(1,13))

diff_third_report_vs_last


In [ ]:
dead_diff_last_report_vs_third_after_month_end

In [ ]:
total_updates = diff_third_report_vs_last.sum()
print (total_updates)
diff_third_report_vs_last[1:11].mean()

In [ ]:
plt.figure(figsize=(18,12))

ax = plt.gca()

for month in monthly_transposed.columns:
    monthly_transposed.plot(ax=ax,y=month,style='o--')
    
plt.xlabel('reporting date')
plt.ylabel('monthly deaths')

legends = [pd.to_datetime(x).month_name() for x in monthly_transposed.columns]

plt.legend(legends)

title1 = 'SWEDEN monthly All Cause Deaths Jan-Dec 2020 - Data reliable 3 weeks after initial report\nDataSource : scb.se'
title2 = "\n\nsee also https://www.scb.se/om-scb/nyheter-och-pressmeddelanden/scb-publicerar-preliminar-statistik-over-doda-i-sverige/"

plt.title(title1+title2)

_= plt.xticks(range(len(monthly_transposed)),monthly_transposed.index,rotation=90)

plt.savefig('swe_deaths_lag.jpg',format='jpg')


In [ ]:
all_diffs_df = pd.DataFrame(all_diffs).T
all_diffs_df.columns = monthly_transposed.columns
all_diffs_df.index = monthly_transposed.index
all_diffs_df

In [ ]:
plt.figure(figsize=(18,12))
plt.title('SWEDEN All Cause Deaths - differences for each monthly death count between latest and earlier reports\nDataSource : scb.se')

for month in all_diffs_df.columns:
    mask = pd.to_datetime(all_diffs_df[month].index) > pd.to_datetime(month) 
    plt.plot (all_diffs_df[month][mask],'o--',label=month.date())
    
plt.legend(loc='upper center')
plt.xlabel('reporting date')
plt.ylabel('difference in registered deaths per reporting date vs latest reported data')
    
_=plt.xticks(rotation=90)